In [1]:
%%html
<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>

<h1 style = "font-size: 30px; text-align: center;">AI MiniMax Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Black Jack</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Sepideh Fatemi</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810896059</h2>
<h4 style="text-align: center">Spring 1400</h4>

در این پروژه با ستفاده از الگوریتم minimax بازی black jack  را پیاده سازی کرده ایم.

در این الگوریتم با استفاده از یک تابع تخمین زن تلاش می کنیم امتیاز بازیکن را زیاد و امتیاز حریف را کم کنیم و در نهایت  از حریف ببریم.
توضیحات این الگوریتم و نحوه پیاده سازی آن در ادامه بیشتر توضیح داده شده است.

In [2]:
import random
from copy import deepcopy

In [3]:
class Player:
    def __init__(self, name, num_of_cards):
        """
        The base player class of the game
        Inputs
        -----------
        name = (str) player's name
        num_of_cards = (int) number of cards in the deck
        """
        self.name = name
        self.deck_count = num_of_cards
        self.target = self.deck_count * 2 - 1
        self.cards = []
        self.erases_remaining = self.deck_count // 5
        self.has_stopped = False

    def draw_card(self, card):
        """
        draws a card, and adds it to player cards
        Input
        -------------
        card: (int) the card to be added
        """
        self.cards.append(card)

    def print_info(self):
        """
        prints info of the player
        """
        print(f"{self.name}'s cards: ", end='')
        for c in self.cards:
            print(f'{c}, ', end='')
        print(f'sum: {sum(self.cards)}')
    
    def get_margin(self):
        """
        returns the margin left to target by the player
        Output
        ----------
        (int) margin to target
        """
        return self.target - sum(self.cards)
    
    def cpu_play(self, seen_cards, deck, enemies_cards):
        """
        The function for cpu to play the game
        Inputs
        ----------
        seen_cards:     (list of ints) the cards that have been seen until now
        deck:           (list of ints) the remaining playing deck of the game
        enemies_cards:  (list of ints) the cards that the enemy currently has.
        Output
        ----------
        (str) a command given to the game
        
        """
        if (len(deck) > 0):
            next_card_in_deck = deck[0]
        else:
            next_card_in_deck = 0
        if (len(deck) > 1):
            next_enemy_card_in_deck = deck[1]
        else:
            next_enemy_card_in_deck = 0
        amount_to_target = self.target - sum(self.cards)
        amount_with_next_card = self.target - (sum(self.cards) + next_card_in_deck)
        enemies_amount_to_target = self.target - sum(enemies_cards)
        enemies_amount_with_next_card = self.target - (sum(enemies_cards) + next_enemy_card_in_deck)
        _stop_condition = amount_to_target < next_card_in_deck and self.erases_remaining <= 0
        _draw_condition_1 = next_card_in_deck != 0
        _draw_condition_2 = amount_with_next_card >= 0
        _erase_condition = self.erases_remaining > 0
        _erase_self_condition = amount_to_target < 0
        _erase_opponent_condition_or = enemies_amount_to_target < (self.target // 7)
        _erase_opponent_condition_or_2 = enemies_amount_with_next_card < (self.target // 7) 
        _erase_opponent_condition_or_3 = enemies_amount_with_next_card <= amount_with_next_card
        _erase_opponent_condition_or_4 = enemies_amount_to_target <= amount_to_target
        _erase_opponent_condition = _erase_opponent_condition_or or _erase_opponent_condition_or_2 or _erase_opponent_condition_or_3
        _erase_opponent_condition = _erase_opponent_condition or _erase_opponent_condition_or_4 
        if (_stop_condition):
            return 'stop'
        elif (_draw_condition_1 and _draw_condition_2):
            return 'draw'
        elif(_erase_self_condition and _erase_condition):
            return 'erase_self'
        elif(_erase_opponent_condition and _erase_condition):
            return 'erase_opponent'
        else:
            return 'stop'
    
    def erase(self, target):
        """
        erases the last card of the target
        Input
        ---------
        target: (Player obj) the player whos last card is about to be erased
        """
        if (len(target.cards) == 0):
#             print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target.cards.pop(-1)
#             print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
#         print(f'{self.name} has no more erases remaining!')

    def get_player_cards(self):
        return self.cards

    def get_erases_remained(self):
        return self.erases_remaining

در کلاس State حالت بازی را در هر مرحله ذخیره می کنیم.
روند کلی این الگوریتم به این صورت است که هر بار ککه نوبت به حرکت player می شود، یک State جدید از کل بازی در نظر می گیریم و با ریشه آن استیت درختی ایجاد می کنیم.

به این صورت که به ازای هر ۴ حرکتی که player می تواند انجام دهد state child آن را ایجاد می کنیم  

۴ استیت اولیه را در همان متد get_player_input  ایجاد می کنیم که در نهایت بتوانیم حرکتی را که بیشترین heuristic  را دارد بازگردانیم.

heuristic: در این تابع اگر کارت های بازیکن از ۴۱ بیشتر باشد امتیاز منفی و اگر کمتر باشد به همان میزان امتیاز مثبت می گیرد  و کارت های حریف اگر از ۴۱ بیشتر باشد امتیاز مثبت برای ما دارد چون  ما می خواهیم حریف ببازد.
از طرفی اگر حریف در استانه برد باشد امتیاز ما باید کم شود.  تفاضل امتیاز بازیکن ها تا ۴۱  یا همان margin را هم به عنوان فاکتوری تاثیرگذار درنظر گرفتم.

In [4]:
class State:
    def __init__(self, player, opponent, deck, turn_player, depth):
        self.player = deepcopy(player)
        self.opponent = deepcopy(opponent)
        self.deck = deepcopy(deck)
        self.turn = turn_player
        self.depth = depth
        self.moves = ['draw', 'stop', 'erase_self', 'erase_opponent']

    def calculate_heuristic(self):
        player_margin = (41-sum(self.player.cards))*2
        opponent_margin = 41-sum(self.player.cards)

        if sum(self.player.cards) > 41:
            player_cards = -sum(self.player.cards) * 2
        else:
            player_cards = sum(self.player.cards) * 3

        if sum(self.opponent.cards) > 41:
            opponent_cards = sum(self.opponent.cards)
        else:
            opponent_cards = -sum(self.opponent.cards)

        heuristic = player_cards + opponent_cards - player_margin - opponent_margin
        return heuristic


در تابع child_state استیت پدر و حرکت به آن داده می شود و استیت جدید را باز می گرداند.  
باید توجه شود که در این تابع باید حواسمان به نوبت باشد و اگر در استیت فعلی نوبت بازیکن است برای استیت بچه نوبت را به حریف بدهیم.

In [5]:
def child_state(state, move):
    child_deck = deepcopy(state.deck)
    if (state.turn == 'player'):
        child_turn = 'opponent'
        opponent = deepcopy(state.opponent)
        player = deepcopy(state.player)
    else:
        child_turn = 'player'
        opponent = deepcopy(state.player)
        player = deepcopy(state.opponent)

    if player.has_stopped is True:
        return False
    if (move == 'stop' or move == 's'):
        if player.has_stopped is False:
            player.has_stopped = True
        else:
            return None
    elif (move == 'draw' or move == 'd'):
        if (len(child_deck) > 0):
            card = child_deck.pop(0)
            player.draw_card(card)
        else:
            opponent.has_stopped = True
            player.has_stopped = True
            return None

    elif ((move == 'erase_self' or move == 'es')):
        if len(player.cards) > 0 and player.erases_remaining > 0:
            player.erases_remaining -= 1
            player.erase(player)
        else:
            return None
    elif ((move == 'erase_opponent' or move == 'eo')):
        if len(opponent.cards) > 0 and player.erases_remaining > 0:
            player.erases_remaining -= 1
            player.erase(opponent)
        else:
            return None
    else:
        print('ERROR: unknown command')
        return False
    if (state.turn == 'player'):
        child = State(player, opponent, child_deck, child_turn, state.depth + 1)
    else:
        child = State(opponent, player, child_deck, child_turn, state.depth + 1)
    return child


In [34]:
class Blacksin:
    def __init__(self, deck_count=21):
        """
        The main game class
        Inputs
        -----------
        deck_count = (int) number of cards in the deck
        """
        self.deck_count = deck_count
        self.target = self.deck_count * 2 - 1
        self.player = Player('player', deck_count)
        self.opponent = Player('opponent', deck_count)
        self.deck = self.shuffle_cards()
        self.seen_cards = []
    
    def shuffle_cards(self):
        """ 
        shuffles cards for deck creation
        """
        return list(random.sample(range(1, self.deck_count + 1), self.deck_count))

    def draw_card(self):
        """ 
        draws a card from deck, if non is remaining, ends the game.
        """
        if (len(self.deck) > 0):
            card = self.deck.pop(0)
            self.seen_cards.append(card)
            return card
        print('The deck is empty! ending game...')
        self.opponent.has_stopped = True
        self.player.has_stopped = True
        return -1

    def handout_cards(self):
        """ 
        hands out cards to players
        """
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
    
    def handle_input(self, _input, player):
        """ 
        handles input
        Input
        ------------
        _input: (str) input given by the player
        player: (Player obj)the player that is giving the input
        
        """
        if (player is self.player):
            opponent = self.opponent
        else:
            opponent = self.player
        if (_input == 'stop' or _input == 's'):
            player.has_stopped = True
            print(f'{player.name} has stopped')
        elif (_input == 'draw' or _input == 'd'):
            card = self.draw_card()
            if (card == -1): return True
            player.draw_card(card)
            print(f'{player.name} drawed a card: {card}')
        elif ((_input == 'erase_self' or _input == 'es')):
            print(f'{player.name} erased from {player.name}\'s deck!')
            player.erase(player)
        elif ((_input == 'erase_opponent' or _input == 'eo')):
            print(f'{player.name} erased from {opponent.name}\'s deck!')
            player.erase(opponent)
        else:
            print('ERROR: unknown command')
            return False
        return True

    def get_player_input(self):
        current_state = State(self.player, self.opponent, self.deck, 'player', 0)
        sim_score = {}
        for move in current_state.moves:
            child = child_state(current_state, move)
            if child is None:
                continue
            child.action = move
#             sim_score[move] = minValue(child)
            sim_score[move] = minValueAlphaBeta(child, -1000, 1000)
        player_input = max(sim_score, key=sim_score.get)
        self.handle_input(player_input, self.player)
            
    def opponent_play(self):
        """
        function for opponent to play it's turn
        """
        try:
            opponent_input = self.opponent.cpu_play(self.seen_cards, self.deck, self.player.cards)
        except:
            opponent_input = 'stop'
        self.handle_input(opponent_input, self.opponent)

    def check_for_winners(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            print('an error has accurred! exiting...')
            exit()

    def print_deck(self):
        """
        prints the current deck of the game
        """
        print('full deck: [top] ', end='')
        for i in self.deck:
            print(i, end=' ')
        print('[bottom]')

    def run(self):
        """
        main function to run the game with
        """
        print('\nstarting game... shuffling... handing out cards...')
        print(f'remember, you are aiming for nearest to: {self.target}')
        self.print_deck()
        self.handout_cards()
        turn = 0
        while (not self.player.has_stopped or not self.opponent.has_stopped):
            if (turn == 0):
                if (not self.player.has_stopped):
                    print('player playing...')
                    self.get_player_input()
                    self.opponent.print_info()
                    self.player.print_info()
                    print()
            else:
                if (not self.opponent.has_stopped):
                    print('opponent playing...')
                    self.opponent_play()
                    self.opponent.print_info()
                    self.player.print_info()
                    print()
            turn = 1 - turn
        print('\nand the winner is...')
        return self.check_for_winners()

# minimax

توابع minValue و maxValue  را مطابق آنچه گفته شد پیاده سازی می کنیم.  
ابتدا بررسی می کنیم که اگر به عمق مورد نظر رسیده و یا بازی تمام شده به معنی رسیدن به برگ است و باید تابع heuristic آن state را برگردانیم. و پس از آن به صورت بازگشتی مقدار heuristic  استیت های میانی و در نتیجه بچه های ریشه محاسبه می شود و حرکتی که بیشترین heuristic  را دارد بازگردانده می شود.  
(در این مرحله باید رسیدن به برنده هم بررسی شود ولی به دلیل این نمی خواهیم محاسبات زمان بر شود حتما از عمق کم تر از زمانی که به برد برسد استفاده می شود.)  
در شرطی که False بودن child بررسی می شود، حالتی است که بازیکن توقف کرده در این صورت آن state برگ است و مقدار آن باید بازگردانده شود.  
در شرطی که None بودن child بررسی می شود حالتی است که  حرکتی که با آن به این state child رسیده ایم حرکت غیرمجازی بوده است. درنتیجه نباید اصلا محاسبه شود و درنظر گرفته نمی شود.

In [ ]:
def minValue(state):
    if state.depth >= 5 or (state.player.has_stopped and state.opponent.has_stopped):
        return state.calculate_heuristic()
    v = +1000
    for move in state.moves:
        child = child_state(state, move)
        if child is False:
            return state.calculate_heuristic()
        if child is None:
            continue
        child.action = move
        v = min(v, maxValue(child))
    return v


def maxValue(state):
    if state.depth >= 5 or (state.player.has_stopped and state.opponent.has_stopped):
        return state.calculate_heuristic()
    v = -1000
    for move in state.moves:
        child = child_state(state, move)
        if child is False:
            return state.calculate_heuristic()
        if child is None:
            continue
        child.action = move
        v = max(v, minValue(child))
    return v

# minmax run

In [30]:
import time
start = time.time()

player_wins = 0
opponent_wins = 0
draw = 0
rounds = 500
for each_round in range(rounds):
    print(f'round {each_round} has started')
    game = Blacksin(deck_count=21)
    result = game.run()
    if result == 1:
        player_wins += 1
    if result == 0:
        draw += 1
    if result == -1:
        opponent_wins += 1
end = time.time()

print('player wins: ', player_wins)
print('opponent wins: ', opponent_wins)
print('draw: ', draw)

print(player_wins/rounds*100 , '% of times player wins.')
print(opponent_wins/rounds*100 , '% of times opponent wins.')
print("end in ", '%.2f' % (end - start), 'sec')


round 0 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 11 5 1 4 13 16 18 15 8 6 9 3 12 21 2 7 20 10 19 17 14 [bottom]
player playing...
player drawed a card: 13
opponent's cards: 5, 4, sum: 9
player's cards: 11, 1, 13, sum: 25

opponent playing...
opponent drawed a card: 16
opponent's cards: 5, 4, 16, sum: 25
player's cards: 11, 1, 13, sum: 25

player playing...
player erased from opponent's deck!
opponent's cards: 5, 4, sum: 9
player's cards: 11, 1, 13, sum: 25

opponent playing...
opponent drawed a card: 18
opponent's cards: 5, 4, 18, sum: 27
player's cards: 11, 1, 13, sum: 25

player playing...
player drawed a card: 15
opponent's cards: 5, 4, 18, sum: 27
player's cards: 11, 1, 13, 15, sum: 40

opponent playing...
opponent drawed a card: 8
opponent's cards: 5, 4, 18, 8, sum: 35
player's cards: 11, 1, 13, 15, sum: 40

player playing...
player drawed a card: 6
opponent's cards: 5, 4, 18, 8, sum: 35
player's c

player drawed a card: 4
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, 5, 7, 4, sum: 37

opponent playing...
opponent erased from player's deck!
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, 5, 7, sum: 33

player playing...
player drawed a card: 18
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, 5, 7, 18, sum: 51

opponent playing...
opponent erased from player's deck!
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, 5, 7, sum: 33

player playing...
player has stopped
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, 5, 7, sum: 33

opponent playing...
opponent erased from player's deck!
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, 5, sum: 26

opponent playing...
opponent erased from player's deck!
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, sum: 21

opponent playing...
opponent has stopped
opponent's cards: 2, 14, 10, sum: 26
player's cards: 21, sum: 21


and the winner is...
opponent's cards: 2, 14, 10, sum: 26
play

player drawed a card: 10
opponent's cards: 8, 1, 13, sum: 22
player's cards: 3, 21, 15, 10, sum: 49

opponent playing...
opponent drawed a card: 19
opponent's cards: 8, 1, 13, 19, sum: 41
player's cards: 3, 21, 15, 10, sum: 49

player playing...
player erased from opponent's deck!
opponent's cards: 8, 1, 13, sum: 22
player's cards: 3, 21, 15, 10, sum: 49

opponent playing...
opponent drawed a card: 6
opponent's cards: 8, 1, 13, 6, sum: 28
player's cards: 3, 21, 15, 10, sum: 49

player playing...
player erased from player's deck!
opponent's cards: 8, 1, 13, 6, sum: 28
player's cards: 3, 21, 15, sum: 39

opponent playing...
opponent erased from player's deck!
opponent's cards: 8, 1, 13, 6, sum: 28
player's cards: 3, 21, sum: 24

player playing...
player drawed a card: 16
opponent's cards: 8, 1, 13, 6, sum: 28
player's cards: 3, 21, 16, sum: 40

opponent playing...
opponent drawed a card: 5
opponent's cards: 8, 1, 13, 6, 5, sum: 33
player's cards: 3, 21, 16, sum: 40

player playing...
pla

player drawed a card: 14
opponent's cards: 15, 17, sum: 32
player's cards: 8, 10, 14, sum: 32

opponent playing...
opponent drawed a card: 3
opponent's cards: 15, 17, 3, sum: 35
player's cards: 8, 10, 14, sum: 32

player playing...
player drawed a card: 11
opponent's cards: 15, 17, 3, sum: 35
player's cards: 8, 10, 14, 11, sum: 43

opponent playing...
opponent drawed a card: 6
opponent's cards: 15, 17, 3, 6, sum: 41
player's cards: 8, 10, 14, 11, sum: 43

player playing...
player erased from opponent's deck!
opponent's cards: 15, 17, 3, sum: 35
player's cards: 8, 10, 14, 11, sum: 43

opponent playing...
opponent erased from player's deck!
opponent's cards: 15, 17, 3, sum: 35
player's cards: 8, 10, 14, sum: 32

player playing...
player erased from opponent's deck!
opponent's cards: 15, 17, sum: 32
player's cards: 8, 10, 14, sum: 32

opponent playing...
opponent erased from player's deck!
opponent's cards: 15, 17, sum: 32
player's cards: 8, 10, sum: 18

player playing...
player drawed a 

player drawed a card: 16
opponent's cards: 18, 9, 8, sum: 35
player's cards: 4, 7, 1, 16, sum: 28

opponent playing...
opponent erased from player's deck!
opponent's cards: 18, 9, 8, sum: 35
player's cards: 4, 7, 1, sum: 12

player playing...
player drawed a card: 10
opponent's cards: 18, 9, 8, sum: 35
player's cards: 4, 7, 1, 10, sum: 22

opponent playing...
opponent has stopped
opponent's cards: 18, 9, 8, sum: 35
player's cards: 4, 7, 1, 10, sum: 22

player playing...
player erased from opponent's deck!
opponent's cards: 18, 9, sum: 27
player's cards: 4, 7, 1, 10, sum: 22

player playing...
player erased from opponent's deck!
opponent's cards: 18, sum: 18
player's cards: 4, 7, 1, 10, sum: 22

player playing...
player erased from opponent's deck!
opponent's cards: sum: 0
player's cards: 4, 7, 1, 10, sum: 22

player playing...
player has stopped
opponent's cards: sum: 0
player's cards: 4, 7, 1, 10, sum: 22


and the winner is...
opponent's cards: sum: 0
player's cards: 4, 7, 1, 10, sum

...

player erased from player's deck!
opponent's cards: 8, 16, 1, sum: 25
player's cards: 18, 13, sum: 31

opponent playing...
opponent erased from player's deck!
opponent's cards: 8, 16, 1, sum: 25
player's cards: 18, sum: 18

player playing...
player erased from player's deck!
opponent's cards: 8, 16, 1, sum: 25
player's cards: sum: 0

opponent playing...
opponent has stopped
opponent's cards: 8, 16, 1, sum: 25
player's cards: sum: 0

player playing...
player drawed a card: 20
opponent's cards: 8, 16, 1, sum: 25
player's cards: 20, sum: 20

player playing...
player drawed a card: 7
opponent's cards: 8, 16, 1, sum: 25
player's cards: 20, 7, sum: 27

player playing...
player drawed a card: 5
opponent's cards: 8, 16, 1, sum: 25
player's cards: 20, 7, 5, sum: 32

player playing...
player drawed a card: 6
opponent's cards: 8, 16, 1, sum: 25
player's cards: 20, 7, 5, 6, sum: 38

player playing...
player has stopped
opponent's cards: 8, 16, 1, sum: 25
player's cards: 20, 7, 5, 6, sum: 38


and 

player's cards: 5, 10, 8, 11, 2, sum: 36


and the winner is...
opponent's cards: 16, 15, sum: 31
player's cards: 5, 10, 8, 11, 2, sum: 36
the winner is the player!
round 485 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 10 7 15 11 20 17 5 19 8 12 16 9 21 1 18 6 4 13 2 14 3 [bottom]
player playing...
player erased from opponent's deck!
opponent's cards: 7, sum: 7
player's cards: 10, 15, sum: 25

opponent playing...
opponent drawed a card: 20
opponent's cards: 7, 20, sum: 27
player's cards: 10, 15, sum: 25

player playing...
player drawed a card: 17
opponent's cards: 7, 20, sum: 27
player's cards: 10, 15, 17, sum: 42

opponent playing...
opponent drawed a card: 5
opponent's cards: 7, 20, 5, sum: 32
player's cards: 10, 15, 17, sum: 42

player playing...
player erased from player's deck!
opponent's cards: 7, 20, 5, sum: 32
player's cards: 10, 15, sum: 25

opponent playing...
opponent has stopped
opponent's card

player drawed a card: 8
opponent's cards: 16, 12, 7, sum: 35
player's cards: 18, 13, 5, 8, sum: 44

opponent playing...
opponent erased from player's deck!
opponent's cards: 16, 12, 7, sum: 35
player's cards: 18, 13, 5, sum: 36

player playing...
player erased from opponent's deck!
opponent's cards: 16, 12, sum: 28
player's cards: 18, 13, 5, sum: 36

opponent playing...
opponent erased from player's deck!
opponent's cards: 16, 12, sum: 28
player's cards: 18, 13, sum: 31

player playing...
player erased from player's deck!
opponent's cards: 16, 12, sum: 28
player's cards: 18, sum: 18

opponent playing...
opponent has stopped
opponent's cards: 16, 12, sum: 28
player's cards: 18, sum: 18

player playing...
player drawed a card: 20
opponent's cards: 16, 12, sum: 28
player's cards: 18, 20, sum: 38

player playing...
player drawed a card: 1
opponent's cards: 16, 12, sum: 28
player's cards: 18, 20, 1, sum: 39

player playing...
player has stopped
opponent's cards: 16, 12, sum: 28
player's car

player drawed a card: 5
opponent's cards: 10, 2, sum: 12
player's cards: 13, 17, 5, sum: 35

opponent playing...
opponent drawed a card: 14
opponent's cards: 10, 2, 14, sum: 26
player's cards: 13, 17, 5, sum: 35

player playing...
player erased from opponent's deck!
opponent's cards: 10, 2, sum: 12
player's cards: 13, 17, 5, sum: 35

opponent playing...
opponent drawed a card: 11
opponent's cards: 10, 2, 11, sum: 23
player's cards: 13, 17, 5, sum: 35

player playing...
player drawed a card: 7
opponent's cards: 10, 2, 11, sum: 23
player's cards: 13, 17, 5, 7, sum: 42

opponent playing...
opponent drawed a card: 1
opponent's cards: 10, 2, 11, 1, sum: 24
player's cards: 13, 17, 5, 7, sum: 42

player playing...
player erased from player's deck!
opponent's cards: 10, 2, 11, 1, sum: 24
player's cards: 13, 17, 5, sum: 35

opponent playing...
opponent drawed a card: 3
opponent's cards: 10, 2, 11, 1, 3, sum: 27
player's cards: 13, 17, 5, sum: 35

player playing...
player drawed a card: 6
oppone

player erased from opponent's deck!
opponent's cards: 5, 6, 11, sum: 22
player's cards: 9, 3, 14, 13, sum: 39

opponent playing...
opponent drawed a card: 7
opponent's cards: 5, 6, 11, 7, sum: 29
player's cards: 9, 3, 14, 13, sum: 39

player playing...
player has stopped
opponent's cards: 5, 6, 11, 7, sum: 29
player's cards: 9, 3, 14, 13, sum: 39

opponent playing...
opponent erased from player's deck!
opponent's cards: 5, 6, 11, 7, sum: 29
player's cards: 9, 3, 14, sum: 26

opponent playing...
opponent has stopped
opponent's cards: 5, 6, 11, 7, sum: 29
player's cards: 9, 3, 14, sum: 26


and the winner is...
opponent's cards: 5, 6, 11, 7, sum: 29
player's cards: 9, 3, 14, sum: 26
the winner is the opponent!
round 499 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 16 20 2 5 14 15 6 3 18 4 12 17 7 11 13 9 21 8 10 19 1 [bottom]
player playing...
player drawed a card: 14
opponent's cards: 20, 5, sum: 25
player's

اگر این روش را با عمق های مختلف بررسی کنیم متوجه می شویم که هر چه عمق را افزایش دهیم زمان اجرای الگوریتم بیشتر می شود ولی دقت آن افزایش می یابد و اگر عمق را حدود ۱۰ درنظر بگیریم بهترین عملکرد را دارد به دلیل این که درخت ما احتمالا تا مرحله ای که به برد ختم می شود را بررسی می کند و در نهایت بهترین حرکت ممکن را باز می گرداند.  
من عمق را ۵ درنظرگرفتم به این دلیل که نسبتا عملکرد خوبی دارد و زمان آن هم خیلی زیاد نمی باشد

# Alpha Beta Pruning

در روش alpha beta pruning در زمان پیمایش درخت مثلا زمانی که داریم بچه های یک نود   minValue را  محاسبه می کنیم به دلیل این که پدر این استیت به دنبال بیشترین heuristic است اگر یکی از این بچه ها، بچه ای با  heuristic کمتر از مقدار قبلی داشت دیگر لازم نیست بچه های آن نود را محاسبه کنیم.

بنابراین بهترین امتیازی که نود max میتواند بگیرد را در apha و بهترین امتیاز نود min  را در beta  ذخیره می کنیم. که بتوانیم در مراحل بعد با مقایسه با این مقادیر درخت را هرس کنیم

به این ترتیب الگوریتم در زمان بسیار کمتری به جواب می رسد  


توجه شود در این روش مقادیر نود های میانی  ممکن است اشتباه باشند. ولی مقدار heuristic برای نود ریشه حتما درست است.

در این روش اگر ترتیب نودها را تغییر دهیم، می دانیم که value ها همان خواهد بود بنابر این تاثیر آن در مقدار هرسی است که در درخت اتفاق می افتد. و درنتیجه در میزان صرفه جویی در زمان تاثیر می گذارد.
پس از بررسی کردن چینش های مختلف به این نتیجه رسیدم که ['draw', 'stop', 'erase_self', 'erase_opponent'] بهترین چینش می باشد و احتمالا به این دلیل است که با توجه به این که تمایل داریم مقدار heuristic  برای بازیکن زیاد و برای playerکم کنیم بنابر این ابتدا حرکتی را انتخاب می کنیم که احتمال زیاد بهتر است و مقدار heuristicرا بیشتر زیاد می کند

In [21]:
def minValueAlphaBeta(state, alpha, beta):
    if state.depth >= 5 or (state.player.has_stopped and state.opponent.has_stopped):
        return state.calculate_heuristic()
    v = +1000
    for move in state.moves:
        child = child_state(state, move)
        if child is False:
            return state.calculate_heuristic()
        if child is None:
            continue
        child.action = move
        v = min(v, maxValueAlphaBeta(child, alpha, beta))
        if v <= alpha:
            return v
        beta = min(beta, v)
    return v


def maxValueAlphaBeta(state, alpha, beta):
    if state.depth >= 5 or (state.player.has_stopped and state.opponent.has_stopped):
        return state.calculate_heuristic()
    v = -1000
    for move in state.moves:
        child = child_state(state, move)
        if child is False:
            return state.calculate_heuristic()
        if child is None:
            continue
        child.action = move
        v = max(v, minValueAlphaBeta(child, alpha, beta))
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v


در تابع get_player_input تابع minValueAlphaBeta را صدا می زنیم.   
توجه شود این دستور کامنت شده است.


# alpha beta run

In [35]:
import time
start = time.time()

player_wins = 0
opponent_wins = 0
draw = 0
rounds = 500
for each_round in range(rounds):
    print(f'round {each_round} has started')
    game = Blacksin(deck_count=21)
    result = game.run()
    if result == 1:
        player_wins += 1
    if result == 0:
        draw += 1
    if result == -1:
        opponent_wins += 1
end = time.time()

print('player wins: ', player_wins)
print('opponent wins: ', opponent_wins)
print('draw: ', draw)

print(player_wins/rounds*100 , '% of times player wins.')
print(opponent_wins/rounds*100 , '% of times opponent wins.')
print("end in ", '%.2f' % (end - start), 'sec')


round 0 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 6 19 16 2 14 12 9 1 13 4 18 8 20 10 11 7 17 3 5 15 21 [bottom]
player playing...
player drawed a card: 14
opponent's cards: 19, 2, sum: 21
player's cards: 6, 16, 14, sum: 36

opponent playing...
opponent drawed a card: 12
opponent's cards: 19, 2, 12, sum: 33
player's cards: 6, 16, 14, sum: 36

player playing...
player drawed a card: 9
opponent's cards: 19, 2, 12, sum: 33
player's cards: 6, 16, 14, 9, sum: 45

opponent playing...
opponent drawed a card: 1
opponent's cards: 19, 2, 12, 1, sum: 34
player's cards: 6, 16, 14, 9, sum: 45

player playing...
player erased from opponent's deck!
opponent's cards: 19, 2, 12, sum: 33
player's cards: 6, 16, 14, 9, sum: 45

opponent playing...
opponent erased from player's deck!
opponent's cards: 19, 2, 12, sum: 33
player's cards: 6, 16, 14, sum: 36

player playing...
player erased from opponent's deck!
opponent's cards

player has stopped
opponent's cards: 14, 8, 3, 4, 11, sum: 40
player's cards: 18, 13, 7, sum: 38

opponent playing...
opponent erased from player's deck!
opponent's cards: 14, 8, 3, 4, 11, sum: 40
player's cards: 18, 13, sum: 31

opponent playing...
opponent has stopped
opponent's cards: 14, 8, 3, 4, 11, sum: 40
player's cards: 18, 13, sum: 31


and the winner is...
opponent's cards: 14, 8, 3, 4, 11, sum: 40
player's cards: 18, 13, sum: 31
the winner is the opponent!
round 6 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 13 21 16 3 18 9 11 2 4 8 15 17 7 12 10 6 20 5 1 14 19 [bottom]
player playing...
player drawed a card: 18
opponent's cards: 21, 3, sum: 24
player's cards: 13, 16, 18, sum: 47

opponent playing...
opponent drawed a card: 9
opponent's cards: 21, 3, 9, sum: 33
player's cards: 13, 16, 18, sum: 47

player playing...
player erased from player's deck!
opponent's cards: 21, 3, 9, sum: 33
player's car

player drawed a card: 6
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, 11, 6, sum: 33

opponent playing...
opponent erased from player's deck!
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, 11, sum: 27

player playing...
player drawed a card: 8
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, 11, 8, sum: 35

opponent playing...
opponent erased from player's deck!
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, 11, sum: 27

player playing...
player erased from player's deck!
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, sum: 16

opponent playing...
opponent has stopped
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, sum: 16

player playing...
player drawed a card: 19
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, 19, sum: 35

player playing...
player drawed a card: 3
opponent's cards: 4, 7, 12, 14, sum: 37
player's cards: 16, 19, 3, sum: 38

player playing...
player erased from opponent's deck!
oppon

player drawed a card: 17
opponent's cards: 9, 5, 3, 15, sum: 32
player's cards: 20, 10, 17, sum: 47

opponent playing...
opponent erased from player's deck!
opponent's cards: 9, 5, 3, 15, sum: 32
player's cards: 20, 10, sum: 30

player playing...
player erased from opponent's deck!
opponent's cards: 9, 5, 3, sum: 17
player's cards: 20, 10, sum: 30

opponent playing...
opponent drawed a card: 19
opponent's cards: 9, 5, 3, 19, sum: 36
player's cards: 20, 10, sum: 30

player playing...
player drawed a card: 1
opponent's cards: 9, 5, 3, 19, sum: 36
player's cards: 20, 10, 1, sum: 31

opponent playing...
opponent erased from player's deck!
opponent's cards: 9, 5, 3, 19, sum: 36
player's cards: 20, 10, sum: 30

player playing...
player drawed a card: 7
opponent's cards: 9, 5, 3, 19, sum: 36
player's cards: 20, 10, 7, sum: 37

opponent playing...
opponent erased from player's deck!
opponent's cards: 9, 5, 3, 19, sum: 36
player's cards: 20, 10, sum: 30

player playing...
player has stopped
opp

player drawed a card: 14
opponent's cards: 11, 16, 8, 4, sum: 39
player's cards: 3, 10, 5, 19, 14, sum: 51

opponent playing...
opponent drawed a card: 2
opponent's cards: 11, 16, 8, 4, 2, sum: 41
player's cards: 3, 10, 5, 19, 14, sum: 51

player playing...
player erased from opponent's deck!
opponent's cards: 11, 16, 8, 4, sum: 39
player's cards: 3, 10, 5, 19, 14, sum: 51

opponent playing...
opponent erased from player's deck!
opponent's cards: 11, 16, 8, 4, sum: 39
player's cards: 3, 10, 5, 19, sum: 37

player playing...
player erased from opponent's deck!
opponent's cards: 11, 16, 8, sum: 35
player's cards: 3, 10, 5, 19, sum: 37

opponent playing...
opponent erased from player's deck!
opponent's cards: 11, 16, 8, sum: 35
player's cards: 3, 10, 5, sum: 18

player playing...
player drawed a card: 15
opponent's cards: 11, 16, 8, sum: 35
player's cards: 3, 10, 5, 15, sum: 33

opponent playing...
opponent has stopped
opponent's cards: 11, 16, 8, sum: 35
player's cards: 3, 10, 5, 15, sum

...

player drawed a card: 15
opponent's cards: 13, 18, sum: 31
player's cards: 21, 1, 15, sum: 37

opponent playing...
opponent drawed a card: 9
opponent's cards: 13, 18, 9, sum: 40
player's cards: 21, 1, 15, sum: 37

player playing...
player drawed a card: 2
opponent's cards: 13, 18, 9, sum: 40
player's cards: 21, 1, 15, 2, sum: 39

opponent playing...
opponent has stopped
opponent's cards: 13, 18, 9, sum: 40
player's cards: 21, 1, 15, 2, sum: 39

player playing...
player erased from opponent's deck!
opponent's cards: 13, 18, sum: 31
player's cards: 21, 1, 15, 2, sum: 39

player playing...
player erased from opponent's deck!
opponent's cards: 13, sum: 13
player's cards: 21, 1, 15, 2, sum: 39

player playing...
player has stopped
opponent's cards: 13, sum: 13
player's cards: 21, 1, 15, 2, sum: 39


and the winner is...
opponent's cards: 13, sum: 13
player's cards: 21, 1, 15, 2, sum: 39
the winner is the player!
round 481 has started

starting game... shuffling... handing out cards...
remem

player drawed a card: 21
opponent's cards: 9, 15, sum: 24
player's cards: 18, 17, 21, sum: 56

opponent playing...
opponent drawed a card: 5
opponent's cards: 9, 15, 5, sum: 29
player's cards: 18, 17, 21, sum: 56

player playing...
player erased from opponent's deck!
opponent's cards: 9, 15, sum: 24
player's cards: 18, 17, 21, sum: 56

opponent playing...
opponent drawed a card: 11
opponent's cards: 9, 15, 11, sum: 35
player's cards: 18, 17, 21, sum: 56

player playing...
player erased from player's deck!
opponent's cards: 9, 15, 11, sum: 35
player's cards: 18, 17, sum: 35

opponent playing...
opponent erased from player's deck!
opponent's cards: 9, 15, 11, sum: 35
player's cards: 18, sum: 18

player playing...
player drawed a card: 13
opponent's cards: 9, 15, 11, sum: 35
player's cards: 18, 13, sum: 31

opponent playing...
opponent drawed a card: 6
opponent's cards: 9, 15, 11, 6, sum: 41
player's cards: 18, 13, sum: 31

player playing...
player drawed a card: 3
opponent's cards: 9, 15

player's cards: 6, 4, 7, 18, sum: 35
the winner is the opponent!
round 489 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 3 7 19 16 13 18 4 11 9 17 15 2 12 1 6 21 20 8 14 5 10 [bottom]
player playing...
player drawed a card: 13
opponent's cards: 7, 16, sum: 23
player's cards: 3, 19, 13, sum: 35

opponent playing...
opponent drawed a card: 18
opponent's cards: 7, 16, 18, sum: 41
player's cards: 3, 19, 13, sum: 35

player playing...
player drawed a card: 4
opponent's cards: 7, 16, 18, sum: 41
player's cards: 3, 19, 13, 4, sum: 39

opponent playing...
opponent erased from player's deck!
opponent's cards: 7, 16, 18, sum: 41
player's cards: 3, 19, 13, sum: 35

player playing...
player erased from opponent's deck!
opponent's cards: 7, 16, sum: 23
player's cards: 3, 19, 13, sum: 35

opponent playing...
opponent drawed a card: 11
opponent's cards: 7, 16, 11, sum: 34
player's cards: 3, 19, 13, sum: 35

player playing.

player erased from opponent's deck!
opponent's cards: 15, sum: 15
player's cards: 19, 16, sum: 35

opponent playing...
opponent drawed a card: 13
opponent's cards: 15, 13, sum: 28
player's cards: 19, 16, sum: 35

player playing...
player drawed a card: 5
opponent's cards: 15, 13, sum: 28
player's cards: 19, 16, 5, sum: 40

opponent playing...
opponent erased from player's deck!
opponent's cards: 15, 13, sum: 28
player's cards: 19, 16, sum: 35

player playing...
player erased from opponent's deck!
opponent's cards: 15, sum: 15
player's cards: 19, 16, sum: 35

opponent playing...
opponent drawed a card: 14
opponent's cards: 15, 14, sum: 29
player's cards: 19, 16, sum: 35

player playing...
player drawed a card: 6
opponent's cards: 15, 14, sum: 29
player's cards: 19, 16, 6, sum: 41

opponent playing...
opponent drawed a card: 11
opponent's cards: 15, 14, 11, sum: 40
player's cards: 19, 16, 6, sum: 41

player playing...
player erased from opponent's deck!
opponent's cards: 15, 14, sum: 29


player erased from player's deck!
opponent's cards: 17, 1, 8, 7, sum: 33
player's cards: 20, sum: 20

opponent playing...
opponent has stopped
opponent's cards: 17, 1, 8, 7, sum: 33
player's cards: 20, sum: 20

player playing...
player drawed a card: 19
opponent's cards: 17, 1, 8, 7, sum: 33
player's cards: 20, 19, sum: 39

player playing...
player erased from opponent's deck!
opponent's cards: 17, 1, 8, sum: 26
player's cards: 20, 19, sum: 39

player playing...
player has stopped
opponent's cards: 17, 1, 8, sum: 26
player's cards: 20, 19, sum: 39


and the winner is...
opponent's cards: 17, 1, 8, sum: 26
player's cards: 20, 19, sum: 39
the winner is the player!
round 497 has started

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 4 9 20 3 2 12 18 16 5 7 14 17 8 11 19 10 15 1 6 21 13 [bottom]
player playing...
player drawed a card: 2
opponent's cards: 9, 3, sum: 12
player's cards: 4, 20, 2, sum: 26

opponent playing...
op

اگر از ترتیب کارت ها اطلاعی نداشتیم عنصر شانس به مسئله اضافه می شد. در این حال محیط دیگر deterministic  نمی باشد و باید از احتمال استفاده کنیم و باید از نود شانس استفاده کنیم. در این حالت باید حالت میانگین را در نظر بگیریم و میانگین هر کدام از action ها را با توجه به احتمال آن محاسبه می کنیم  و آن را به عنوان امتیاز نهایی هر نود شانس انتخاب می کنیم. 
و الگوریتم minimax  احتمالا جواب خوبی به ما نمی دهد.

بنابر این اگر از روش minimaxبا عمق مناسب استفاده کنیم به احتمال زیاد از حریفی که با عمق یک بازی  می کند می بریم و برای افزایش سرعت و صرفه جویی در زمان از روش  هرس کردن شاخه هایی که مطُمئن هستیم در جواب تاثیری ندارند استفاده میکنیم.

In [ ]:
import os
os.system('jupyter nbconvert --to html AI_Handson_game.ipynb')